## ДЗ 1:
### Разделиться на команды по 4 человека и выбрать модели для оптимизации, сделать базовые замеры метрик производительности

In [1]:
import ultralytics
from ultralytics import YOLO

In [2]:
import sys
import os

import torch
import time
import torch.quantization
import torch
from torch import profiler
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
torch.seed = 42

In [4]:
def get_model_size(model: torch.nn.Module) -> float:
    """return model size im mb"""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    return (param_size + buffer_size) / 1024**2

In [16]:
def profile_model(model, input_example: torch.Tensor = torch.randn(1, 3, 224, 224)):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_example = input_example.to(device)

    # cold run
    for _ in range(3):
        model(input_example)

    with profile(activities=[ProfilerActivity.CPU], profile_memory=True, with_flops=True) as prof:
        with record_function("model_inference"):
            with torch.inference_mode():
                model(input_example)

    profiling_info = prof.key_averages().table(sort_by="cpu_time_total", row_limit=1)
    return profiling_info

### Исходная модель

In [17]:
yolo = YOLO('yolov8x.pt')
model = yolo.model
model.fuse()
model.eval()
print()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [18]:
print(profile_model(model))
print(f'model size: {get_model_size(model)}')

STAGE:2023-10-09 21:10:02 34666:1731361 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        -7.45%  -60447.000us       100.00%     811.066ms     811.066ms           0 b     -81.30 Mb             1            --  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 811.066ms

model size: 260.1646728515625


STAGE:2023-10-09 21:10:03 34666:1731361 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-09 21:10:03 34666:1731361 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
